In [32]:
!pip install agno
!pip install mem0ai

from agno.agent import Agent
from mem0 import MemoryClient

In [34]:
from agno.agent import Agent
from mem0 import MemoryClient
from transformers import pipeline

# Initialize the Mem0 memory client with API key
memory = MemoryClient(api_key="Mem0-API-KEY")

# Load a lightweight language model for text generation
llm = pipeline("text-generation", model="distilgpt2")

# Agent for storing user input into Mem0 memory
class UpdateAgent(Agent):
    def run(self, input_text):
        memory.add(messages=[{"role": "user", "content": input_text}], user_id="user123")# Static user ID for this demo
        return " Data stored successfully."

# Agent for retrieving most relevant memory from Mem0
class RetrievalAgent(Agent):
    def run(self, query):
        results = memory.search(query=query, user_id="user123", limit=1)
        if results:
            return f" Memory found: {results[0]['memory']}"
        return " No relevant memory found."

# Agent demonstrating agent chaining logic
class ChainedAgent(Agent):
    def run(self, query):
        result = retrieval_agent.run(query)
        return f" Agent chain output:\n→ Retrieved: {result}\n→ Final decision: Log complete."

# Smart agent using both memory context and LLM to generate enhanced answers
class SmartRetrievalAgent(Agent):
    def run(self, query):
        results = memory.search(query=query, user_id="user123", limit=1)
        if results:
            context = results[0]['memory']
            # Prompt engineering: ask the LLM using context
            response = llm(f"Q: {query} | Context: {context}", max_new_tokens=50)
            return f" Smart answer: {response[0]['generated_text'].strip()}"
        return " No smart answer found."

# Agent instances
update_agent = UpdateAgent(name="UpdateAgent")
retrieval_agent = RetrievalAgent(name="RetrievalAgent")
chained_agent = ChainedAgent(name="ChainedAgent")
smart_agent = SmartRetrievalAgent(name="SmartRetrievalAgent")

#  CLI Loop
print("===  Mem0 + Agno Interactive Agent ===")
print("Commands: 'store', 'ask', 'smart', or 'exit'")

while True:
    cmd = input("\nCommand: ").strip().lower()

    if cmd == "store":
        content = input(" What should I remember? ")
        print(update_agent.run(content))

    elif cmd == "ask":
        query = input(" Ask something: ")
        print(retrieval_agent.run(query))

    elif cmd == "smart":
        query = input(" Ask something and get a smart LLM response: ")
        print(smart_agent.run(query))

    elif cmd == "exit":
        print(" Exiting.")
        break

    else:
      # validation for unsupported commands
        print("---- Unknown command. Use 'store', 'ask', 'smart', or 'exit'.")


Device set to use cpu


===  Mem0 + Agno Interactive Agent ===
Commands: 'store', 'ask', 'smart', or 'exit'

Command: store
 What should I remember? i have a meeting tomorrow at 2.30p.m. and also have a coursework submission at 12.00 a.m. on day after tomorrow sunday
 Data stored successfully.

Command: ask
 Ask something: when is my meeting?
 Memory found: Has a meeting tomorrow at 2:30 p.m.

Command: smart
 Ask something and get a smart LLM response: do i have anything related to my academics?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Smart answer: Q: do i have anything related to my academics? | Context: Has a coursework submission at 12:00 a.m. day after tomorrow Sunday? | Context: Will the day be marked as a "no" or "yes" during this week's post? What do you think of this post? | Context: Has a postwork submission at 11:00 a.m. day after

Command: exit
 Exiting.
